<H1> Règles d'association </H1>

Scikit learn est très pauvre pour la recherche de patterns. Il existe cependant quelques librairies qui ont été développées.
La recherche de patterns nécessite souvent des algorithmes très performants pour trouver les résultats. Il est donc courant de devoir 
passer par une API vers un code développé en C++ par exemple. Nous présentons ici deux approches différentes.  
La première utilise une librairie développée en Python qui permet de pouvoir extraire des itemsets et des règles d'association. Il s'agit de la librairie MLxtend : http://rasbt.github.io/mlxtend/.  

La seconde utilise une implémentation d'un algorithme de recherche de règles d'association développé en Java. Il s'agit de SPMF. SPMF propose un très grand choix d'algorithmes pour extraire des règles d'association, des motifs séquentiels, des sous graphes. Il propose également un grand nombre de jeu de données. Il est très utilisé dans la communauté Data Mining : http://www.philippe-fournier-viger.com/spmf/. 

** Rappel de la problématique de la recherche de règles d'association**

Soit ${\displaystyle \mathrm {I} =\left\{i_{1},i_{2},...,i_{m}\right\}}$ un ensemble d'items. Soit ${\displaystyle \mathrm {T} =\left\{t_{1},t_{2},...,t_{n}\right\}}$ un ensemble de transactions, telles que $t_i$ soit un sous-ensemble de $I$ (i.e. $t_i  \subseteq I$). Une règle d'association s'exprime sous la forme :  ${\displaystyle \mathrm {X} \rightarrow \mathrm{Y}}$, où 
${\displaystyle \mathrm {X} \in \mathrm {T} ,\mathrm{Y}\in \mathrm {T} ,}$ et 
${\displaystyle \mathrm {X} \cap \mathrm {Y}\neq \emptyset }$.   

Initialement la problématique de la recherche de règles d'association etait définie par : A partir d'une base de données de transactions, l'objectif est d'extraire l'ensemble des règles qui sont telles que le support de la règle est $\geq$ support_minimal et la confiance (*confidence*) est $\geq$ confiance_minimale où  support_minimal confiance_minimale sont deux paramètres définis par l'utilisateur final. Cependant au cours des années différentes mesures ont été proposées notamment pour remplacer la confiance.


Le *Support* d'une règle ${\displaystyle \mathrm {X} \rightarrow \mathrm{Y}}$, noté ${\displaystyle Support({\displaystyle \mathrm {X,Y}} )}$ indique la fréquence de la règle dans les transactions, i.e. combien de fois les items ${\displaystyle \mathrm {X}}$ et ${\displaystyle \mathrm {Y}}$ apparaissent ensemble, i.e ${\displaystyle Support({\displaystyle \mathrm {X} \cup \mathrm {Y}} )}$.  

La *Confidence* d'une règle ${\displaystyle \mathrm {X} \rightarrow \mathrm{Y}}$ est définie par : 
${\displaystyle Confidence({\displaystyle \mathrm {X} \rightarrow \mathrm{Y}} )}$= ${\displaystyle p(Y|X)}$ = ${\displaystyle \frac{support(X,Y)}{suppport(X)}}$. Range = [0,1]  

Différentes mesures ont été proposées dans la littérature.    



${\displaystyle Lift(\mathrm {X} \rightarrow \mathrm{Y}}$ ) ${\displaystyle= \frac{p(Y|X)}{p(Y)}}$=${\displaystyle \frac{suppport(X,Y)}{suppport(X)\times suppport(Y)}}$ = ${\displaystyle \frac{Confidence({\displaystyle \mathrm {X} \rightarrow \mathrm{Y}} )}{support(X)}}$. Range  [0,$\infty$]
 
Si des règles ont un lift de 1 cela voudrait dire que la probabilité de l'antécédent et du conséquent sont indépendantes l'une de l'autres. Quand deux événements sont indépendants il est donc difficile d'en tirer une règle. Si le lift est supérieur à 1 cela montre que les deux parties sont dépendantes et qu'une règle est peut être utile pour prédire le conséquent à partir de l'antécédent.   



${\displaystyle Leverage(\mathrm {X} \rightarrow \mathrm{Y}}$ ) =${\displaystyle support(X,Y)−support(A)×support(C)}$. Range = [-1,1]  

Une valeur de leverage nulle indique que X et Y sont indépendantes.  
 
${\displaystyle Conviction(\mathrm {X} \rightarrow \mathrm{Y})}$=${\displaystyle \frac{1 -suppport(Y)}{1-Confidence({\displaystyle \mathrm {X} \rightarrow \mathrm{Y})}}}$. Range [-1,1]  

Une valeur de conviction de 1 indique que les événements sont indépendants. 



## Utilisation de  MLxtend

Dans l'exemple nous considérons l'exercice fait en cours. Les données sont organisées de la manière suivante :  
    Chaque ligne correspond à un client, les items achetés sont précisés.

In [ ]:
basket = [['Biscuit', 'Creme', 'Couches', 
            'Pain', 'Confiture'],
           ['Oeufs', 'Salade', 'Pain'],
           ['Pain', 'Biscuit', 'Couches', 
            'Lait','Camembert','Biere','Poudre de cacao'],
           ['Biere', 'Lait', 'Creme', 
            'Pain', 'Salade','Couches','Poudre de cacao'],
           ['Camembert', 'Confiture', 
            'Poudre de cacao', 'Lait'],
           ['Lait','Biere','Perrier',
            'Oeufs','Couches'],
           ['Biscuit', 'Creme', 'Oeufs', 
            'Pain', 'Couches','Lait',
            'Poudre de cacao', 'Confiture'],
           ['Camembert','Biere','Perrier',
            'Oeufs','Salade','Confiture','Couches'],
           ['Salade','Confiture','Pain','Oeufs',
            'Creme','Perrier','Biscuit']]


Il faut à présent transformer les données pour qu'elles correspondent à une matrice où les items sont les colonnes, les clients les lignes et faire apparaître un 0 ou un 1 lorsqu'un client a acheté ou pas un item.   

mlxtend propose une fonction TransactionEncoder pour réaliser cette opération.

In [ ]:

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(basket).transform(basket)
df = pd.DataFrame(te_ary, columns=te.columns_)


Pour connaître l'ensemble des itemsets fréquents il suffit d'appeler l'algorithme apriori.

In [ ]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(df, min_support=0.3, use_colnames=True)

display(frequent_itemsets.head(10))

L'obtention des règles se fait par la fonction association rules. Il est possible de spécifier différentes metrique (Cf. remarque précédente).

In [ ]:
from mlxtend.frequent_patterns import association_rules
print ('Utilisation de la confiance\n')
rules = association_rules(frequent_itemsets, 
                          metric="confidence", 
                          min_threshold=0.2)

display(rules.sample(10))

In [ ]:
print ('Utilisation du lift\n')
rules = association_rules(frequent_itemsets, 
                          metric="lift")

display(rules.head(10))

Le code suivant permet de pouvoir ajouter une colonne qui contient la taille de la partie antecedent pour ne pouvoir afficher que les règles qui ont un certain nombre d'items dans la partie antécédent. 

In [ ]:

rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))


Affichage des règles dont la partie antécédent est supérieure à 1.

In [ ]:
display (rules.loc[(rules["antecedent_len"]>1)].head(10))

## Utilisation d'une API extérieure (SPMF) 

Le code java de SPMF est disponible ici :   
     http://www.philippe-fournier-viger.com/spmf/index.php?link=download.php  
Il faut le télécharger et sauvegarder le fichier jar dans le répertoire courant.  

Comme vous pouvez le constater il existe un très grand nombre d'algorithmes de recherche de règles d'association disponibles. Nous allons utiliser ici FP_Growth.   

Pour obtenir les itemsets : FPGrowth_itemsets  
Pour obtenir les règles d'association : FPGrowth_association_rules  
Pour obtenir les règles d'association avec le lift : FPGrowth_association_rules_with_lift

Dans l'exemple nous utiliserons le dernier. Se reporter à la page de documentation pour connaître les différents paramètres. 

L'appel se fait simplement via :  
java -jar spmf.jar run FPGrowth_association_rules_with_lift NomduFichierFormatSPMF Sortie.txt support% confiance% valeurlift"  

Dans l'exemple nous prenons le fichier contextIGB.txt, support 50%, confiance 90%, lift 1.2.




In [ ]:
import os
    
os.system("java -jar spmf.jar run FPGrowth_association_rules_with_lift Dataset/contextIGB.txt output.txt 50% 90% 1.2")




In [ ]:
#Affichage des résultats
f=open("output.txt")
for ln in f:
    display (ln)
f.close()    